In [20]:
import gym
import random
import numpy as np
import time
# import tensorflow as tf
from IPython.display import clear_output

In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
class Agent():
  def __init__(self, env):

    self.is_discrete = type(env.action_space) == gym.spaces.discrete.Discrete

    if self.is_discrete:
      self.action_size = env.action_space.n

    else:
      self.action_low = env.action_space.low
      self.action_high = env.action_space.high
      self.action_shape = env.action_space.shape
  
  def get_action(self, state):

    if self.is_discrete:
      action = random.choice(range(self.action_size))

    else:
      action = np.random.uniform(self.action_low, self.action_high, self.action_shape)

    return action

In [29]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate

        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())
        
        self.build_model()
        
    def build_model(self):
        tf.reset_default_graph()
        # self.session.reset_default_graph()
        self.state_in =  tf.placeholder(shape=[1], dtype=tf.int32)
        self.action_in = tf.placeholder(shape=[1], dtype=tf.int32)
        self.target_in = tf.placeholder(shape=[1], dtype=tf.float32)

        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)

        self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)

        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.session.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = ([exp] for exp in experience)
        
        q_next = self.session.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros([self.action_size])
        q_target = reward + self.discount_rate * np.max(q_next)
        
        # q_update = q_target - self.q_table[state,action]
        # self.q_table[state,action] += self.learning_rate * q_update

        feed = {self.state_in: state, self.action_in: action, self.target_in: q_target}
        self.session.run(self.optimizer, feed_dict=feed)
        
        if experience[4]:
            self.eps = self.eps * 0.99

    def __del__(self):
        self.session.close()

In [32]:
env = gym.make('FrozenLake-v1', is_slippery=False)
agent = QNAgent(env)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

State size: 16
Observation space: Discrete(16)
Action space: Discrete(4)


/tmp/ipykernel_733918/3944015681.py:26: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  self.q_state = tf.layers.dense(self.state, units=self.action_size, name="q_table")


In [ ]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()

        with tf.variable_scope("q_table", reuse=True):
            weights = agent.session.run(tf.get_variable("kernel"))
            print(weights)
            
        time.sleep(0.05)
        clear_output(wait=True)